# Hyperelasticity
Author: Jørgen S. Dokken and Garth N. Wells

This section shows how to solve the hyperelasticity problem for deformation of a beam.

We will also show how to create a constant boundary condition for a vector function space.

We start by importing DOLFINx and some additional dependencies.
Then, we create a slender cantilever consisting of hexahedral elements and create the function space `V` for our unknown.

In [1]:
import numpy as np
import ufl

from petsc4py import PETSc
from mpi4py import MPI
from dolfinx import fem, mesh, plot
L = 20.0
domain = mesh.create_box(MPI.COMM_WORLD,[[0.0,0.0,0.0], [L, 1, 1]], [20, 5, 5], mesh.CellType.hexahedron)
V = fem.VectorFunctionSpace(domain, ("CG", 2))

We create two python functions for determining the facets to apply boundary conditions to

In [2]:
def left(x):
    return np.isclose(x[0], 0)

def right(x):
    return np.isclose(x[0], L)

fdim = domain.topology.dim -1
left_facets = mesh.locate_entities_boundary(domain, fdim, left)
right_facets = mesh.locate_entities_boundary(domain, fdim, right)

Next, we create a  marker based on these two functions

In [3]:
# Concatenate and sort the arrays based on facet indices. Left facets marked with 1, right facets with two
marked_facets = np.hstack([left_facets, right_facets])
marked_values = np.hstack([np.full(len(left_facets), 1, dtype=np.int32), np.full(len(right_facets), 2, dtype=np.int32)])
sorted_facets = np.argsort(marked_facets)
facet_tag = mesh.meshtags(domain, fdim, marked_facets[sorted_facets], marked_values[sorted_facets])

We then create a function for supplying the boundary condition on the left side, which is fixed.

In [4]:
u_bc = np.array((0,) * domain.geometry.dim, dtype=PETSc.ScalarType)

To apply the boundary condition, we identity the dofs located on the facets marked by the `MeshTag`.

In [5]:
left_dofs = fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.indices[facet_tag.values==1])
bcs = [fem.dirichletbc(u_bc, left_dofs, V)]

Next, we define the body force on the reference configuration (`B`), and nominal (first Piola-Kirchhoff) traction (`T`). 

In [6]:
B = fem.Constant(domain, PETSc.ScalarType((0, 0, 0)))
T = fem.Constant(domain, PETSc.ScalarType((0, 0, 0)))

Define the test and solution functions on the space $V$

In [7]:
v = ufl.TestFunction(V)
u = fem.Function(V)

Define kinematic quantities used in the problem

In [8]:
# Spatial dimension
d = len(u)

# Identity tensor
I = ufl.variable(ufl.Identity(d))

# Deformation gradient
F = ufl.variable(I + ufl.grad(u))

# Right Cauchy-Green tensor
C = ufl.variable(F.T * F)

# Invariants of deformation tensors
Ic = ufl.variable(ufl.tr(C))
J  = ufl.variable(ufl.det(F))

Define the elasticity model via a stored strain energy density function $\psi$, and create the expression for the first Piola-Kirchhoff stress:

In [9]:
# Elasticity parameters
E = PETSc.ScalarType(1.0e4)
nu = PETSc.ScalarType(0.3)
mu = fem.Constant(domain, E/(2*(1 + nu)))
lmbda = fem.Constant(domain, E*nu/((1 + nu)*(1 - 2*nu)))
# Stored strain energy density (compressible neo-Hookean model)
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J))**2
# Stress
# Hyper-elasticity
P = ufl.diff(psi, F)

```{admonition} Comparison to linear elasticity
To illustrate the difference between linear and hyperelasticity, the following lines can be uncommented to solve the linear elasticity problem.
```

In [10]:
# P = 2.0 * mu * ufl.sym(ufl.grad(u)) + lmbda * ufl.tr(ufl.sym(ufl.grad(u))) * I

Define the variational form with traction integral over all facets with value 2. We set the quadrature degree for the integrals to 4.

In [10]:
metadata = {"quadrature_degree": 4}
ds = ufl.Measure('ds', domain=domain, subdomain_data=facet_tag, metadata=metadata)
dx = ufl.Measure("dx", domain=domain, metadata=metadata)
# Define form F (we want to find u such that F(u) = 0)
F = ufl.inner(ufl.grad(v), P)*dx - ufl.inner(v, B)*dx - ufl.inner(v, T)*ds(2) 

As the varitional form is non-linear and written on residual form, we use the non-linear problem class from DOLFINx to set up required structures to use a Newton solver.

In [11]:
problem = fem.petsc.NonlinearProblem(F, u, bcs)

and then create and customize the Newton solver

In [12]:
from dolfinx import nls
solver = nls.petsc.NewtonSolver(domain.comm, problem)

# Set Newton solver options
solver.atol = 1e-8
solver.rtol = 1e-8
solver.convergence_criterion = "incremental"

We create a function to plot the solution at each time step.

In [15]:
import pyvista
pyvista.set_jupyter_backend("ipygany")

grid = pyvista.UnstructuredGrid(*plot.create_vtk_mesh(domain, domain.topology.dim))

def plot_function(t, uh):
    """
    Create a figure of the concentration uh warped visualized in 3D at timet step t.
    """
    p = pyvista.Plotter()
    # Create grid defined by the function space for visualization of the function
    topology, cells, geometry = plot.create_vtk_mesh(uh.function_space)
    function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)
    var_name = f"u({t})"
    values = np.zeros((geometry.shape[0], 3))
    values[:, :len(uh)] = uh.x.array.reshape(geometry.shape[0], len(uh))
    function_grid[var_name] = values
    function_grid.set_active_vectors(var_name)
    # Warp mesh by deformation
    warped = function_grid.warp_by_vector(var_name, factor=1)
    
    # Add mesh to plotter and visualize
    actor = p.add_mesh(warped)
    p.show_axes()
    if not pyvista.OFF_SCREEN:
       p.show()
    else:
        pyvista.start_xvfb()
        figure_as_array = p.screenshot(f"diffusion_{t:.2f}.png")
        # Clear plotter for next plot
        p.remove_actor(actor)

plot_function(0, u)

AppLayout(children=(VBox(children=(HTML(value='<h3>u(0)</h3>'), Dropdown(description='Colormap:', options={'Br…

Finally, we solve the problem over several time steps, updating the y-component of the traction

In [17]:
from dolfinx import log
log.set_log_level(log.LogLevel.INFO)
tval0 = -1.5
for n in range(1, 10):
    T.value[2] = n * tval0
    num_its, converged = solver.solve(u)
    assert(converged)
    u.x.scatter_forward()
    print(f"Time step {n}, Number of iterations {num_its}, Load {T.value}")
    # plot_function(n, u)

2022-03-31 12:58:01.537 ( 282.296s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-03-31 12:58:02.370 ( 283.130s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-03-31 12:58:02.912 ( 283.671s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 29.7635 (tol = 1e-08) r (rel) = 0.148549(tol = 1e-08)
2022-03-31 12:58:03.156 ( 283.915s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-03-31 12:58:03.793 ( 284.552s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 3: r (abs) = 25.3713 (tol = 1e-08) r (rel) = 0.126628(tol = 1e-08)
2022-03-31 12:58:04.054 ( 284.814s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-03-31 12:58:04.663 ( 285.423s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 4

Time step 1, Number of iterations 30, Load [ 0.   0.  -1.5]


2022-03-31 12:58:26.370 ( 307.129s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-03-31 12:58:27.193 ( 307.952s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-03-31 12:58:27.824 ( 308.584s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 17.3254 (tol = 1e-08) r (rel) = 0.117842(tol = 1e-08)
2022-03-31 12:58:28.200 ( 308.959s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-03-31 12:58:28.788 ( 309.547s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 3: r (abs) = 5.14882 (tol = 1e-08) r (rel) = 0.0350207(tol = 1e-08)
2022-03-31 12:58:29.018 ( 309.777s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-03-31 12:58:29.651 ( 310.410s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 

Time step 2, Number of iterations 9, Load [ 0.  0. -3.]


2022-03-31 12:58:33.773 ( 314.532s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-03-31 12:58:34.583 ( 315.342s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-03-31 12:58:35.174 ( 315.934s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 10.0011 (tol = 1e-08) r (rel) = 0.0887471(tol = 1e-08)
2022-03-31 12:58:35.441 ( 316.200s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-03-31 12:58:36.014 ( 316.773s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 3: r (abs) = 5.33026 (tol = 1e-08) r (rel) = 0.0472992(tol = 1e-08)
2022-03-31 12:58:36.241 ( 317.000s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-03-31 12:58:36.767 ( 317.526s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration

...

2022-03-31 12:59:06.341 ( 347.100s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-03-31 12:59:07.025 ( 347.785s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 6: r (abs) = 1.78868e-08 (tol = 1e-08) r (rel) = 4.34715e-10(tol = 1e-08)
2022-03-31 12:59:07.025 ( 347.785s) [main            ]       NewtonSolver.cpp:255   INFO| Newton solver finished in 6 iterations and 6 linear solver iterations.
2022-03-31 12:59:07.253 ( 348.012s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-03-31 12:59:08.112 ( 348.872s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-03-31 12:59:08.650 ( 349.409s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 1.06336 (tol = 1e-08) r (rel) = 0.031085(tol = 1e-08)
2022-03-31 12:59:08.890 ( 349.649s) [main            ]              petsc.cp

Time step 8, Number of iterations 6, Load [  0.   0. -12.]


2022-03-31 12:59:12.347 ( 353.106s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-03-31 12:59:13.201 ( 353.960s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-03-31 12:59:13.765 ( 354.525s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 0.898789 (tol = 1e-08) r (rel) = 0.0309666(tol = 1e-08)
2022-03-31 12:59:14.039 ( 354.798s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-03-31 12:59:14.641 ( 355.401s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 3: r (abs) = 1.38354 (tol = 1e-08) r (rel) = 0.0476679(tol = 1e-08)
2022-03-31 12:59:14.869 ( 355.628s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.


Time step 9, Number of iterations 6, Load [  0.    0.  -13.5]


2022-03-31 12:59:15.486 ( 356.245s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 4: r (abs) = 0.00185096 (tol = 1e-08) r (rel) = 6.37724e-05(tol = 1e-08)
2022-03-31 12:59:15.740 ( 356.500s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-03-31 12:59:16.267 ( 357.026s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 5: r (abs) = 7.87183e-06 (tol = 1e-08) r (rel) = 2.71213e-07(tol = 1e-08)
2022-03-31 12:59:16.497 ( 357.256s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-03-31 12:59:17.047 ( 357.807s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 6: r (abs) = 4.57091e-13 (tol = 1e-08) r (rel) = 1.57484e-14(tol = 1e-08)
2022-03-31 12:59:17.047 ( 357.807s) [main            ]       NewtonSolver.cpp:255   INFO| Newton solver finished in 6 iterations and 6 linear solver iterations.
